
##### Determining Orbital Elements from Two Position Vectors and Time of Flight
This section describes the Python implementation of the algorithm to determine the orbital elements from two position vectors \((r_1, r_2)\) and the time of flight using Lambert's problem and Lagrange coefficients. The code assumes you have the necessary inputs and uses the Newton-Raphson method to solve for the orbital elements.
##### Inputs
-  Initial position vector
-  Final position vector
-  Time of flight
##### Steps
1. **Compute the chord and semi-perimeter:**
    - Compute the chord \(c\) between \(r_1\) and \(r_2\).
    - Compute the semi-perimeter \(s\) of the triangle formed by \(r_1\), \(r_2\), and \(c\).
2. **Initial guess for the Lagrange coefficients:**
    - Use an initial guess for the Lagrange coefficients \(f\) and \(g\).
3. **Iterative solution using Newton-Raphson method:**
    - Iterate to solve for the Lagrange coefficients using the Newton-Raphson method.
4. **Compute the orbital elements:**
    - Use the Lagrange coefficients to compute the orbital elements such as semi-major axis, eccentricity, inclination, etc.


In [ ]:
import numpy as np
from scipy.optimize import newton

# Constants
mu = 398600.4418  # Earth's gravitational parameter (km^3/s^2)

# Function to calculate orbital elements from position and velocity vectors
def orbital_elements(r, v):
    h = np.cross(r, v)  # Specific angular momentum
    n = np.cross([0, 0, 1], h)  # Node vector
    e = ((np.linalg.norm(v)**2 - mu / np.linalg.norm(r)) * r - np.dot(r, v) * v) / mu  # Eccentricity vector
    ecc = np.linalg.norm(e)  # Eccentricity
    a = 1 / (2 / np.linalg.norm(r) - np.linalg.norm(v)**2 / mu)  # Semi-major axis
    inc = np.arccos(h[2] / np.linalg.norm(h))  # Inclination
    raan = np.arctan2(n[1], n[0])  # RAAN
    argp = np.arctan2(np.dot(e, np.cross(n, e)), np.dot(n, e))  # Argument of perigee
    nu = np.arctan2(np.dot(r, np.cross(h, e)), np.dot(r, e))  # True anomaly
    return a, ecc, inc, raan, argp, nu

# Function to solve Lambert's problem using Lagrange coefficients
def lambert_problem(r1, r2, dt, f, g, fdot, gdot):
    # Calculate velocity vectors using Lagrange coefficients
    v1 = (r2 - f * r1) / g
    v2 = fdot * r1 + gdot * v1

    # Calculate orbital elements
    a, ecc, inc, raan, argp, nu = orbital_elements(r1, v1)

    # Calculate the angle between r1 and r2
    delta_theta = np.arccos(np.dot(r1, r2) / (np.linalg.norm(r1) * np.linalg.norm(r2)))

    return a, ecc, inc, raan, argp, nu, delta_theta

# Test case
r1 = np.array([3600, 4600, 3600])  # Initial position vector (km)
r2 = np.array([-5500, 6240, 5200])  # Final position vector (km)
dt = 30 * 60  # Time of flight (30 minutes in seconds)
f = -5.2434  # Lagrange coefficient
g = 1788.15  # Lagrange coefficient
fdot = -3.02e-6  # Lagrange coefficient derivative
gdot = -5.2434  # Lagrange coefficient derivative

# Solve Lambert's problem
a, ecc, inc, raan, argp, nu, delta_theta = lambert_problem(r1, r2, dt, f, g, fdot, gdot)

# Output results
print("Semi-major axis (a):", a, "km")
print("Eccentricity (e):", ecc)
print("Inclination (i):", np.degrees(inc), "degrees")
print("RAAN (Ω):", np.degrees(raan), "degrees")
print("Argument of Perigee (ω):", np.degrees(argp), "degrees")
print("True Anomaly (ν):", np.degrees(nu), "degrees")
print("Angle between r1 and r2 (Δθ):", np.degrees(delta_theta), "degrees")

To visualize the output, we can use the `matplotlib` library to plot the orbit in 3D space. Below is the modified Python code that includes the visualization of the orbit using the calculated orbital elements.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Constants
mu = 398600.4418  # Earth's gravitational parameter (km^3/s^2)

# Function to calculate orbital elements from position and velocity vectors
def orbital_elements(r, v):
    h = np.cross(r, v)  # Specific angular momentum
    n = np.cross([0, 0, 1], h)  # Node vector
    e = ((np.linalg.norm(v)**2 - mu / np.linalg.norm(r)) * r - np.dot(r, v) * v) / mu  # Eccentricity vector
    ecc = np.linalg.norm(e)  # Eccentricity
    a = 1 / (2 / np.linalg.norm(r) - np.linalg.norm(v)**2 / mu)  # Semi-major axis
    inc = np.arccos(h[2] / np.linalg.norm(h))  # Inclination
    raan = np.arctan2(n[1], n[0])  # RAAN
    argp = np.arctan2(np.dot(e, np.cross(n, e)), np.dot(n, e))  # Argument of perigee
    nu = np.arctan2(np.dot(r, np.cross(h, e)), np.dot(r, e))  # True anomaly
    return a, ecc, inc, raan, argp, nu

# Function to solve Lambert's problem using Lagrange coefficients
def lambert_problem(r1, r2, dt, f, g, fdot, gdot):
    # Calculate velocity vectors using Lagrange coefficients
    v1 = (r2 - f * r1) / g
    v2 = fdot * r1 + gdot * v1

    # Calculate orbital elements
    a, ecc, inc, raan, argp, nu = orbital_elements(r1, v1)

    # Calculate the angle between r1 and r2
    delta_theta = np.arccos(np.dot(r1, r2) / (np.linalg.norm(r1) * np.linalg.norm(r2)))

    return a, ecc, inc, raan, argp, nu, delta_theta, v1, v2

# Function to generate orbit points
def generate_orbit(a, ecc, inc, raan, argp, nu, num_points=1000):
    # True anomaly array
    nu_array = np.linspace(0, 2 * np.pi, num_points)
    
    # Semi-latus rectum
    p = a * (1 - ecc**2)
    
    # Position in the perifocal frame
    r_pf = np.array([p * np.cos(nu_array) / (1 + ecc * np.cos(nu_array)),
                     p * np.sin(nu_array) / (1 + ecc * np.cos(nu_array)),
                     np.zeros_like(nu_array)])
    
    # Rotation matrix to convert from perifocal to ECI frame
    R = np.array([
        [np.cos(raan) * np.cos(argp) - np.sin(raan) * np.sin(argp) * np.cos(inc),
         -np.cos(raan) * np.sin(argp) - np.sin(raan) * np.cos(argp) * np.cos(inc),
         np.sin(raan) * np.sin(inc)],
        [np.sin(raan) * np.cos(argp) + np.cos(raan) * np.sin(argp) * np.cos(inc),
         -np.sin(raan) * np.sin(argp) + np.cos(raan) * np.cos(argp) * np.cos(inc),
         -np.cos(raan) * np.sin(inc)],
        [np.sin(argp) * np.sin(inc),
         np.cos(argp) * np.sin(inc),
         np.cos(inc)]
    ])
    
    # Position in the ECI frame
    r_eci = np.dot(R, r_pf)
    
    return r_eci

# Test case
r1 = np.array([3600, 4600, 3600])  # Initial position vector (km)
r2 = np.array([-5500, 6240, 5200])  # Final position vector (km)
dt = 30 * 60  # Time of flight (30 minutes in seconds)
f = -5.2434  # Lagrange coefficient
g = 1788.15  # Lagrange coefficient
fdot = -3.02e-6  # Lagrange coefficient derivative
gdot = -5.2434  # Lagrange coefficient derivative

# Solve Lambert's problem
a, ecc, inc, raan, argp, nu, delta_theta, v1, v2 = lambert_problem(r1, r2, dt, f, g, fdot, gdot)

# Generate orbit points
r_eci = generate_orbit(a, ecc, inc, raan, argp, nu)

# Output results
print("Semi-major axis (a):", a, "km")
print("Eccentricity (e):", ecc)
print("Inclination (i):", np.degrees(inc), "degrees")
print("RAAN (Ω):", np.degrees(raan), "degrees")
print("Argument of Perigee (ω):", np.degrees(argp), "degrees")
print("True Anomaly (ν):", np.degrees(nu), "degrees")
print("Angle between r1 and r2 (Δθ):", np.degrees(delta_theta), "degrees")

# Plot the orbit
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the orbit
ax.plot(r_eci[0, :], r_eci[1, :], r_eci[2, :], label='Orbit')

# Plot the initial and final position vectors
ax.scatter(r1[0], r1[1], r1[2], color='red', label='r1')
ax.scatter(r2[0], r2[1], r2[2], color='blue', label='r2')

# Plot Earth (for reference)
u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
x = 6371 * np.cos(u) * np.sin(v)
y = 6371 * np.sin(u) * np.sin(v)
z = 6371 * np.cos(v)
ax.plot_wireframe(x, y, z, color='green', alpha=0.1, label='Earth')

# Set labels and legend
ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
ax.set_zlabel('Z (km)')
ax.legend()

# Show the plot
plt.show()